EDA on predictions

In [1]:
# Import libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# For reproducibility
np.random.seed(42)

Reading in the predictions and the meta data csv files

In [2]:
preds = pd.read_csv('../predictions.csv')

In [3]:
preds.head()

,Percentages,filename,true classes,predicted classes
0,0.399443,84512.jpg,1,0
1,0.150568,99442.jpg,1,0
2,0.098132,75956.jpg,1,0
3,0.442577,7486.jpg,1,0
4,0.544760,35766.jpg,1,1


In [4]:
info = pd.read_csv('../Data_csv/binary_all_info.csv')

In [5]:
info.head()

,style,new_filename,title,source,size_bytes,pixelsy,pixelsx,genre,date,artist
0,Romanticism,84512.jpg,Procession in Naples,wikiart,7896792.0,7352.0,6238.0,cityscape,NaN,Franz Richard Unterberger
1,Romanticism,99442.jpg,Storm at sea,wikiart,3189597.0,5811.0,7444.0,marina,1873,Ivan Aivazovsky
2,Romanticism,75956.jpg,The Constitution of the 3rd May 1791,wikipedia,7284812.0,4135.0,7459.0,history painting,1891.0,Jan Matejko
3,Romanticism,7486.jpg,Death of Eleazer,wikiart,29811319.0,6068.0,4915.0,religious painting,1866,Gustave Dore
4,Romanticism,35766.jpg,The-Deluge,wikiart,32443139.0,6219.0,4770.0,religious painting,NaN,Gustave Dore


First cast Percentages column as actual percentages. This column is how likely something was the positive class.

In [6]:
preds.Percentages = (preds.Percentages * 100).astype(float)

In [7]:
preds.head()

,Percentages,filename,true classes,predicted classes
0,39.944258,84512.jpg,1,0
1,15.056804,99442.jpg,1,0
2,9.813249,75956.jpg,1,0
3,44.257727,7486.jpg,1,0
4,54.476047,35766.jpg,1,1


In [12]:
preds = preds.round(2)

In [13]:
preds.head()

,Percentages,filename,true classes,predicted classes
0,39.94,84512.jpg,1,0
1,15.06,99442.jpg,1,0
2,9.81,75956.jpg,1,0
3,44.26,7486.jpg,1,0
4,54.48,35766.jpg,1,1


In [14]:
info2 = info.rename(columns={'new_filename': 'filename'})

In [15]:
df = pd.merge(preds, info2, on='filename', how='outer')

In [16]:
df.head()

,Percentages,filename,true classes,predicted classes,style,title,source,size_bytes,pixelsy,pixelsx,genre,date,artist
0,39.94,84512.jpg,1.0,0.0,Romanticism,Procession in Naples,wikiart,7896792.0,7352.0,6238.0,cityscape,NaN,Franz Richard Unterberger
1,15.06,99442.jpg,1.0,0.0,Romanticism,Storm at sea,wikiart,3189597.0,5811.0,7444.0,marina,1873,Ivan Aivazovsky
2,9.81,75956.jpg,1.0,0.0,Romanticism,The Constitution of the 3rd May 1791,wikipedia,7284812.0,4135.0,7459.0,history painting,1891.0,Jan Matejko
3,44.26,7486.jpg,1.0,0.0,Romanticism,Death of Eleazer,wikiart,29811319.0,6068.0,4915.0,religious painting,1866,Gustave Dore
4,54.48,35766.jpg,1.0,1.0,Romanticism,The-Deluge,wikiart,32443139.0,6219.0,4770.0,religious painting,NaN,Gustave Dore


In [17]:
df.isnull().sum()

Percentages          14946
filename                 0
true classes         14946
predicted classes    14946
style                    0
title                    0
source                   0
size_bytes               0
pixelsy                  0
pixelsx                  0
genre                   46
date                  5109
artist                   0
dtype: int64

In [18]:
#dropping the 14946 null values since it was in the training set.
df.dropna(subset=['Percentages'], inplace=True)

In [19]:
# checking amount of null values again
df.isnull().sum()

Percentages             0
filename                0
true classes            0
predicted classes       0
style                   0
title                   0
source                  0
size_bytes              0
pixelsy                 0
pixelsx                 0
genre                  12
date                 1077
artist                  0
dtype: int64

In [20]:
# we have 1077 null values from date, and 12 from genre.
# i will input the missing values with 0 for the date.

In [21]:
df['date'].fillna(0, inplace = True)

In [22]:
df['date'].value_counts()

0         1077
1872        77
1882        74
1874        71
1880        70
          ... 
1926         1
c.1863       1
1823.0       1
c.1832       1
1940.0       1
Name: date, Length: 390, dtype: int64

In [23]:
df['Year'] = df['date'].str.extract('(\d+)', expand=False)

In [24]:
df.head(3)

,Percentages,filename,true classes,predicted classes,style,title,source,size_bytes,pixelsy,pixelsx,genre,date,artist,Year
0,39.94,84512.jpg,1.0,0.0,Romanticism,Procession in Naples,wikiart,7896792.0,7352.0,6238.0,cityscape,0,Franz Richard Unterberger,NaN
1,15.06,99442.jpg,1.0,0.0,Romanticism,Storm at sea,wikiart,3189597.0,5811.0,7444.0,marina,1873,Ivan Aivazovsky,1873
2,9.81,75956.jpg,1.0,0.0,Romanticism,The Constitution of the 3rd May 1791,wikipedia,7284812.0,4135.0,7459.0,history painting,1891.0,Jan Matejko,1891


In [25]:
df['Year'].fillna(1870, inplace = True)

In [26]:
df['Year'].value_counts()

1870    1077
1880      94
1882      88
1872      87
1885      85
        ... 
1772       1
1927       1
1770       1
1798       1
1929       1
Name: Year, Length: 167, dtype: int64

In [27]:
df.to_csv('../final_clean2.csv', index=False) 

In [ ]:
sns.pairplot(df, hue="predicted classes", palette="husl")